In [1]:
# downloding catboost
!pip install -qU catboost

In [2]:
# imports
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import accuracy_score

In [3]:
# loading dataset
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
# getting info of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# dropping categorical features with nulls
df.drop(columns=['embarked', 'embark_town', 'deck'], axis=1, inplace=True)

In [6]:
# checking missing values; nulls in numeric features will be handled by catboost
df.isnull().sum() 

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
fare            0
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [7]:
# creating X & y
X = df.drop(columns=['survived'], axis=1)
y = df['survived']

In [8]:
# splitting X & y into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# selecting categorical features
cat = df.select_dtypes(['object', 'category']).columns.to_list()

In [10]:
# model training
model = cb.CatBoostClassifier(n_estimators=500, learning_rate=0.03, loss_function='Logloss', random_state=42, 
                              nan_mode='Min', bootstrap_type='Bayesian')
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], cat_features=cat, early_stopping_rounds=50, verbose=0)

In [11]:
# model evaluation
print('Training Accuracy:', (accuracy_score(y_train, model.predict(X_train)) * 100).round(2))
print('Testing Accuracy:', (accuracy_score(y_test, model.predict(X_test)) * 100).round(2))

Training Accuracy: 100.0
Testing Accuracy: 100.0
